# Imports

In [2]:
import numpy as np
import pandas as pd
import pickle
from generate_data import DataGenerator
import os

# Test Case Summary

In [3]:
# read test cases from file using pickle
with open('simulated_data/test_cases.pkl', 'rb') as f:
    test_cases = pickle.load(f)

In [4]:
# create an overview over all test cases using a dataframe
# we need the cluster ids, the mean positions and stds of each cluster in the test
# also the number of samples in each cluster and the mean velocities and stds

# create a dataframe for the test cases
data_generator = DataGenerator()
df_test_cases = pd.DataFrame()
for i, test_case in enumerate(test_cases):
    for labels, mu_position, mu_std in zip(test_case['clusters'], test_case['mu_position'], test_case['mu_std']):
        mu_velocity = data_generator.df_infos[['U', 'V', 'W']].iloc[labels].values
        std = np.concatenate((mu_std, data_generator.df_infos[['U_std', 'V_std', 'W_std']].iloc[labels].values))
        N = max(int(data_generator.df_infos[['n_cluster']].iloc[labels].values), 200)
        df_test_cases = pd.concat([df_test_cases, pd.DataFrame({
            'test_case': i,
            'cluster': labels,
            'mu_position': [mu_position],
            'mu_std': [mu_std],
            'N': [N],
            'mu_velocity': [mu_velocity],
            'std': [std]
        })])

In [4]:
# calculate difference between the positions, velocities and stds
for i in df_test_cases['test_case'].unique():
    for col in ['mu_position', 'mu_velocity', 'std']:
        positions = df_test_cases.loc[df_test_cases['test_case'] == i, col].apply(np.array)
        diff = np.diff(positions.values, axis=0)
        df_test_cases.loc[df_test_cases['test_case'] == i, f'diff_{col}'] = np.linalg.norm(diff[0])

In [9]:
df_test_cases.head()

,test_case,cluster,mu_position,mu_std,N,mu_velocity,std,diff_mu_position,diff_mu_velocity,diff_std
0,0,0,"[-500, -500, 20]","[3, 5, 2]",305,"[-9.778308, -9.736123, -5.0372443]","[3, 5, 2, 2, 2, 2]",20.615528,2.630699,3.162278
0,0,1,"[-490, -490, 35]","[4, 8, 2]",438,"[-11.969043, -8.369388, -4.5339603]","[4, 8, 2, 2, 2, 2]",20.615528,2.630699,3.162278
0,1,0,"[20, 40, 150]","[7, 3, 4]",305,"[-9.778308, -9.736123, -5.0372443]","[7, 3, 4, 2, 2, 2]",0.000000,12.713389,2.449490
0,1,2,"[20, 40, 150]","[6, 5, 5]",200,"[-20.984257, -14.273282, -8.970582]","[6, 5, 5, 2, 2, 2]",0.000000,12.713389,2.449490
0,2,0,"[-20, 40, 160]","[5, 7, 3]",305,"[-9.778308, -9.736123, -5.0372443]","[5, 7, 3, 2, 2, 2]",20.000000,6.661291,2.236068


# NMI Overview

In [60]:
dir = '/home/nico/Desktop/test_case/'

nmi = pd.DataFrame()
for subdir in os.listdir(dir):
    if os.path.isdir(dir + subdir):
        nmi_sub = pd.read_csv(dir + subdir + '/nmi.csv', index_col=0)
        # tranpose
        nmi_sub = nmi_sub.T
        nmi_sub['test_case'] = int(subdir[9:])
        # get rid of first column
        nmi = pd.concat([nmi, nmi_sub])

nmi = nmi.sort_values(by='test_case')
nmi = nmi.set_index('test_case')
nmi = nmi.round(3)
nmi



strategy,sigma,ttest,bootstrap_range_test,bootstrap_difference_test,xd_mean_distance,xd_sample_ttest,xd_mean_distance_sample_distance,xd_sample_bootstrap_range_test
test_case,,,,,,,,
1,0.302,0.322,0.323,0.303,0.303,0.303,0.303,0.303
2,0.872,0.839,0.872,0.872,0.872,0.872,0.872,0.872
3,0.020,0.080,0.080,0.007,0.081,0.080,0.081,0.007
4,0.145,0.543,0.012,0.029,0.543,0.012,0.015,0.015
5,0.780,0.816,0.816,0.816,0.816,0.816,0.816,0.816
6,0.683,0.473,0.473,0.509,0.509,0.509,0.509,0.509
7,0.389,0.486,0.578,0.695,0.695,0.695,0.695,0.695
8,0.012,0.410,0.085,0.015,0.015,0.025,0.015,0.015
9,0.769,0.856,0.856,0.856,0.856,0.793,0.856,0.856


# Test Case Evaluation

In [22]:
# read all files from the subdirectories starting in split_ using pickle
dir = '/home/nico/Desktop/test_case/'
split_info = {}
for subdir in os.listdir(dir):
    if os.path.isdir(dir + subdir):
        split_info[int(subdir[9:])] = {}
        for file in os.listdir(dir + subdir):
            if file.startswith('split_'):
                key = file[11:-4]
                split_info[int(subdir[9:])][key] = pickle.load(open(dir + subdir + '/' + file, 'rb'))

In [23]:
test_case_overview = pd.DataFrame()
noise_split_overview = pd.DataFrame()

for test_case in split_info:
    for test_strat in split_info[test_case]:
        noise_splits = {
            'count': 0,
            'same_velocity': 0,
            'deviation_overwrite': 0,
            'remerge': 0,
        }
        for split in split_info[test_case][test_strat]:

            true_labels_new = split['labels_nc_id']
            true_labels_old = split['labels_part_of_old_cluster']

            split_type = ''
            velocity_test_output = ' '.join(split['test_output'].split(" ")[:2])
            same_velocity = False if velocity_test_output == 'different velocity' else True
            deviation_overwrite = True if split['test_output'].split(" ")[-1] == 'overwrite' else False
            if np.all(true_labels_new == -1) and np.all(true_labels_old == -1):
                continue
            elif np.all(true_labels_new == -1) or np.all(true_labels_old == -1):
                noise_splits['count'] += 1
                if same_velocity:
                    noise_splits['same_velocity'] += 1
                if deviation_overwrite:
                    noise_splits['deviation_overwrite'] += 1
                if split['same_velocity']:
                    noise_splits['remerge'] += 1
                split_type = 'noise'
            else:
                try:
                    if (split['count_part_of_old_cluster'][1] >  split['count_nc_id'][1] and 
                        split['count_part_of_old_cluster'][2] <  split['count_nc_id'][2]) or \
                        (split['count_part_of_old_cluster'][1] <  split['count_nc_id'][1] and 
                        split['count_part_of_old_cluster'][2] >  split['count_nc_id'][2]):
                        split_type = 'different-cluster-split'
                    else:
                        split_type = 'same-cluster-split'
                except:
                    split_type = 'cluster'

            runtime = split['runtime'] if 'runtime' in split else 0

            test_case_overview = pd.concat([test_case_overview, pd.DataFrame({
                'test_case': int(test_case),
                'test_strat': test_strat,
                'remerge': split['same_velocity'],
                'same_velocity': same_velocity,
                'deviation_overwrite': deviation_overwrite,
                'labels_nc_id': [split['labels_nc_id']],
                'count_nc_id': [split['count_nc_id']],
                'labels_part_of_old_cluster': [split['labels_part_of_old_cluster']],
                'count_part_of_old_cluster': [split['count_part_of_old_cluster']],
                'split_type': [split_type],
                'runtime': runtime
            })])

        noise_split_overview = pd.concat([noise_split_overview, pd.DataFrame({
            'test_strat': [test_strat],
            'count': noise_splits['count'],
            'same_velocity': noise_splits['same_velocity'],
            'deviation_overwrite': noise_splits['deviation_overwrite'],
            'remerge': noise_splits['remerge']
        })], ignore_index=True)

In [16]:
test_strats = test_case_overview['test_strat'].unique()
test_strats

array(['ttest', 'xd_sample_bootstrap_range_test', 'xd_mean_distance',
       'bootstrap_difference_test', 'xd_sample_ttest',
       'xd_mean_distance_sample_distance', 'bootstrap_range_test'],
      dtype=object)

In [10]:
test_strats = [
    'ttest', 
    'bootstrap_range_test',
    'bootstrap_difference_test', 
    'xd_mean_distance', 
    'xd_sample_ttest',
    'xd_mean_distance_sample_distance', 
    'xd_sample_bootstrap_range_test', 
]
split_types = [
    'different-cluster-split', 
    'same-cluster-split', 
    'noise'
]
test_strat = test_strats[6]

for split_type in split_types:
    test_ = test_case_overview[
                (test_case_overview['test_strat'] == test_strat)   & 
                (test_case_overview['split_type'] == split_type) & 
                (test_case_overview['test_case'] != 21)
            ]

    print(f'Same velocity count: {test_["same_velocity"].sum()}, tests: {test_["same_velocity"].count()}')

Same velocity count: 0, tests: 7
Same velocity count: 2, tests: 173
Same velocity count: 0, tests: 75


In [14]:
split_type = 'noise'
test_strat = test_strats[0]

for test_strat in test_strats:
    print(f'Test Strat: {test_strat}')
    test_ = test_case_overview[
                (test_case_overview['test_strat'] == test_strat)   & 
                (test_case_overview['split_type'] != split_type) & 
                (test_case_overview['test_case'] != 21)
            ]

    print(f'    Accuracy: {1 - test_["deviation_overwrite"].sum()/test_["deviation_overwrite"].count()}')

Test Strat: ttest
    Accuracy: 0.23118279569892475
Test Strat: bootstrap_range_test
    Accuracy: 0.27419354838709675
Test Strat: bootstrap_difference_test
    Accuracy: 0.27956989247311825
Test Strat: xd_mean_distance
    Accuracy: 0.043010752688172005
Test Strat: xd_sample_ttest
    Accuracy: 0.053763440860215006
Test Strat: xd_mean_distance_sample_distance
    Accuracy: 0.037634408602150504
Test Strat: xd_sample_bootstrap_range_test
    Accuracy: 0.0


In [32]:
# check the runtime
mean_runtime = test_case_overview[test_case_overview['test_case'] == 21].groupby('test_strat')['runtime'].mean()

# get the number of splits
split_count = test_case_overview[test_case_overview['test_case'] == 21].groupby('test_strat')['runtime'].count()

# get the sum
split_count = test_case_overview[test_case_overview['test_case'] == 21].groupby('test_strat')['runtime'].sum()

In [30]:
mean_runtime

test_strat
bootstrap_difference_test            7.254088
bootstrap_range_test                 7.710453
ttest                               41.549248
xd_mean_distance                     0.518615
xd_mean_distance_sample_distance     0.634444
xd_sample_bootstrap_range_test       0.582427
xd_sample_ttest                      0.513785
Name: runtime, dtype: float64

In [33]:
split_count

test_strat
bootstrap_difference_test           1414.547189
bootstrap_range_test                1495.827905
ttest                               8060.554173
xd_mean_distance                     101.129994
xd_mean_distance_sample_distance     123.716559
xd_sample_bootstrap_range_test       113.573213
xd_sample_ttest                      100.188025
Name: runtime, dtype: float64